# グローバーのアルゴリズム
グローバーのアルゴリズムは未整序のデータの検索が高速にできるというアルゴリズムです。マーキングの部分にあたるオラクルと呼ばれる関数部分に検索のアルゴリズムを入れましょう。

全体の流れは下記の通りです。

１. 重ね合わせ  
２. 探したいデータのマーキング  
３. マーキングを可視化する振幅増幅反転  

マーキングと振幅増幅反転を適切な回数繰り返すと精度が上がります。こちらをBlueqatで実装してみたいと思います。

# インストール

In [3]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## マーキング
2量子ビットのGroverアルゴリズムを考えます。2量子ビットの組み合わせの場合の数は、00,01,10,11の4通りです。その中から特定の組み合わせを検索したいと思います。  
それを実現するにはゲート操作をつかって、「解に対応する状態ベクトルだけに-1がかかる対角行列」を数学的に作ります。  
ここではHゲート、Sゲート、CZゲートを用いれば可能です。

各回路を一つ一つ見ていきましょう。.run_with_sympy_unitary()を実行することで、回路のユニタリ行列を確認することができます。  
CZゲートの対角行列から始めて、それを変更しながら回路を作っていきます。

In [5]:
from blueqat import Circuit

In [6]:
'''
#marking on 11

-------*-----
-------Z-----
'''

Circuit(2).cz[0,1].run_with_sympy_unitary()

Matrix([
[1, 0, 0,  0],
[0, 1, 0,  0],
[0, 0, 1,  0],
[0, 0, 0, -1]])

In [7]:
'''
#marking on 01
 
----S--*--S---
-------Z-------
'''

Circuit(2).s[0].cz[0,1].s[0].run_with_sympy_unitary()

Matrix([
[1,  0, 0, 0],
[0, -1, 0, 0],
[0,  0, 1, 0],
[0,  0, 0, 1]])

In [8]:
'''
#marking on 10
 
--------*------
----S--Z--S---
'''

Circuit(2).s[1].cz[0,1].s[1].run_with_sympy_unitary()

Matrix([
[1, 0,  0, 0],
[0, 1,  0, 0],
[0, 0, -1, 0],
[0, 0,  0, 1]])

In [9]:

'''
#00
 
----S--*--S--
----S--Z--S--
'''

Circuit(2).s[:].cz[0,1].s[:].run_with_sympy_unitary()

Matrix([
[1,  0,  0,  0],
[0, -1,  0,  0],
[0,  0, -1,  0],
[0,  0,  0, -1]])

最後の結果は符号が反転しています。このような場合には、すべてのマイナス記号をプラスに、プラス記号をマイナスに反転させるグローバル位相を考えることができます。

## 振幅増幅反転
振幅増幅反転は、対角項が $\frac{1}{2}$、非対角項が $-\frac{1}{2}$ の行列を用意しマーキングした回路にかけることで、マーキングしたものの振幅が増幅されます。振幅増幅反転のユニタリ変換は各パターン共通となっています。こちらをBlueqatに直してみます。

In [12]:
'''
--H-X-*-X-H--
--H-X-Z-X-H--
'''

Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].run_with_sympy_unitary()

Matrix([
[ 1/2, -1/2, -1/2, -1/2],
[-1/2,  1/2, -1/2, -1/2],
[-1/2, -1/2,  1/2, -1/2],
[-1/2, -1/2, -1/2,  1/2]])

## 回路の実装
では、実際の回路の実装です。

In [4]:
from blueqat import Circuit

#振幅増幅反転
a = Circuit(2).h[:].x[:].cz[0,1].x[:].h[:].m[:]

'''
#00回路
--H--S--*--S----H-X-*-X-H--
--H--S--Z--S----H-X-Z-X-H--
'''

(Circuit(2).h[:].s[:].cz[0,1].s[:] + a).run(shots=100)

Counter({'00': 100})

In [5]:
'''
#01回路
--H-----*-------H-X-*-X-H--
--H--S--Z--S---H-X-Z-X-H--
'''

(Circuit(2).h[:].s[1].cz[0,1].s[1] + a).run(shots=100)

Counter({'01': 100})

In [6]:
'''
#10回路
--H--S--*--S----H-X-*-X-H--
--H-----Z--------H-X-Z-X-H--
'''
(Circuit(2).h[:].s[0].cz[0,1].s[0] + a).run(shots=100)

Counter({'10': 100})

In [7]:
'''
#11回路
--H-----*-------H-X-*-X-H--
--H-----Z-------H-X-Z-X-H--
'''
(Circuit(2).h[:].cz[0,1] + a).run(shots=100)

Counter({'11': 100})

このようにできました。Groverのアルゴリズムが身近になりました。

# アルゴリズムの説明

## Oracle
まずはこのアルゴリズムに出てくるオラクルから説明します。
簡単にいうと、ある入力に対し 0 あるいは 1 を返す関数です。
例として、物を買う状況を考えましょう。お肉屋さんで買える物を 1、 買えない物を 0 とします。

すると以下の状況になります。

$$
f(x) = \begin{cases}
   1\ \  (x\ がお肉屋さんで買える。)\\
    0\ \  (x\ がお肉屋さんで買えない。)
  \end{cases}
$$

これを用いるとパソコンはお肉屋さんで買えなく、牛肉は買えるので、

$$
f(パソコン) = 0,\ \ \ \ f(牛肉) = 1
$$

となります。

このように 0 と 1 に分ける関数をオラクルと言います。

今回は 1 つだけ 1 を返す Oracle を考えます。
つまり $f$ の入力が 00...00 から 11...11 の $2^n$ 個でどこか一つだけ 1 を返し、それ以外は全て 0 を返すもとします。

$$
f(x) = \begin{cases}
   1\ \  (x= \omega)\\
    0\ \  (x\neq \omega)
  \end{cases}
$$

$x = \omega$ で 1 を返すとしています。

### Amplitude amplification (振幅増幅)
振幅という言葉が出てきました。
各状態の係数のことを**振幅**、確率のことを**確率振幅**と言います。

以下の式を見てみます。

$$
H \lvert 0 \rangle \otimes H \lvert 0 \rangle = \frac{1}{2} (\lvert 00 \rangle + \lvert 01 \rangle + \lvert10 \rangle + \lvert 11 \rangle) \xrightarrow{\text{振幅増幅}} \lvert 00 \rangle
$$

この場合、矢印の左側では確率振幅は全て $1/4$ となり、
右側の状態は $00$ の確率振幅が $1$ となっています。

Amplitude amplification は言葉の通りこの確率振幅を上のように増幅させるアルゴリズムです。

以下の図を用いて具体的な計算を説明します。

<img width="30%" src="https://upload.wikimedia.org/wikipedia/commons/1/16/Grovers_algorithm_geometry.png">

参考: https://en.wikipedia.org/wiki/Grover%27s_algorithm

まずは記号の説明をします。
$s$ は全ビットの重ね合わせです。

$$
\lvert s \rangle = \otimes^n  H  \lvert 0 \rangle =\frac{1}{\sqrt{2^n}}\sum^{2^n}_{x\in \{0, 1\}^n}\lvert x \rangle
$$

このうち、振幅増幅させたい状態を $\omega$ としています。

$$
\lvert \omega \rangle = \frac{1}{\sqrt{2^n}} \lvert 00...010...00\rangle
$$

$x$ 番目に1が入っているとします。

$s'$ は $s$ から $\omega$ を除いたベクトルです。

$$
\lvert s' \rangle = \lvert s \rangle - \lvert \omega \rangle = \lvert \omega^{\perp} \rangle
$$

$\omega$ に垂直なことがわかります。

次に上の図のように $s'$ と $\omega$ から成る単位円を考えます。
ただし $s'$ と $\omega$ の長さが違うので

$$
\tilde{\lvert s' \rangle} = \sqrt{\frac{2^n}{2^n-1}} \lvert s' \rangle 
$$
$$
\tilde{\lvert \omega \rangle} = \sqrt{2^n} \lvert \omega \rangle
$$

として長さを共に 1 にします。

このことから、単位円上のベクトル $\psi$ は以下のように表せます。

$$
\lvert \psi \rangle = \cos\phi \tilde{\lvert s' \rangle} + \sin\phi \tilde{\lvert \omega \rangle}
$$

$U_{\omega}$ は $s'$ を軸に $\psi$ を反転させる行列です。
すなわち $\psi$ を $-\phi$ 回転させます。

$$
U_{\omega} \lvert \psi \rangle = \cos(-\phi) \tilde{\lvert s' \rangle} + \sin(-\phi) \tilde{\lvert \omega \rangle} = \cos(\phi) \tilde{\lvert s' \rangle} - \sin(\phi) \tilde{\lvert \omega \rangle}
$$

$U_s$ は $\psi$ を $s$ を軸に反転させる行列です。

$$
U_s \lvert \psi \rangle = \cos \bigg\{ \frac{\theta}{2} + \big(\frac{\theta}{2} - \phi \big) \bigg\} \tilde{\lvert s' \rangle} + \sin \bigg\{ \frac{\theta}{2} + \big(\frac{\theta}{2} - \phi \big) \bigg\}\tilde{\lvert \omega \rangle} = \cos(\theta - \phi)\tilde{\lvert s' \rangle} + \sin(\theta - \phi) \tilde{\lvert \omega \rangle}
$$

#### 概要
アルゴリズムの概要は以下のようになります。

1 $s$ を $U_{\omega}$ を用いて $s'$ で折り返す。

2 $U_{\omega}s$ を $U_s$ を用いて $s$ で折り返す。

以上の流れを詳しく説明していきます。

#### 1. $s'$ に関する折り返し
$s$ について上の図のように $\theta$ を用いて

$$
\lvert s \rangle = \cos\bigl(\frac{\theta}{2}\bigr) \tilde{\lvert s' \rangle} - \sin\bigl(\frac{\theta}{2}\bigr) \tilde{\lvert \omega \rangle}
$$

と定義します。

このとき

$$
\cos\bigl( \frac{\theta}{2} \bigr) = \sqrt{\frac{2^n-1}{2^n}},\ \ \ \  \sin\bigl( \frac{\theta}{2} \bigr) = \sqrt{\frac{1}{2^n}}
$$

と表せます。

$U_{\omega}$ で $s$ を $s'$ を軸に折り返します。
上の図から以下のようにかけます。

$$
U_{\omega} \lvert s \rangle = \cos\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert \omega \rangle} = \cos\bigl(\frac{\theta}{2}\bigr)\tilde{\lvert s' \rangle} - \sin\bigl(\frac{\theta}{2}\bigr)\tilde{\lvert \omega \rangle}
$$

この操作に関しては $\omega$ のみに作用しているので $U_{\omega}$ は上で述べた Oracle を表していることがわかります。

#### 2. $s$ に関する折り返し
$U_s$ で $U_{\omega}s$ を $s$ を軸に折り返します。

$$
U_s U_{\omega} \lvert s\rangle = U_s\biggl( \cos\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(-\frac{\theta}{2}\bigr)\tilde{\lvert \omega \rangle}  \biggr)
$$

ここで $2\theta$ 回転させれば良いので

$$
U_s U_{\omega} \lvert s\rangle =  \cos\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert \omega \rangle}
$$

具体的に $\cos$ と $\sin$ を求めると加法定理から

$$
\cos \frac{3}{2}\theta = \bigl( 1-\frac{4}{2^n} \bigr) \sqrt{\frac{2^n-1}{2^n}},\ \ \ \ \sin \frac{3}{2}\theta = \bigl( 3-\frac{4}{2^n} \bigr) \sqrt{\frac{1}{2^n}}
$$

よって、$s'$, $\omega$ を用いると

$$
U_s U_{\omega} \lvert s\rangle =  \bigl(1 - \frac{4}{2^n}\bigr)\lvert s' \rangle + \bigl(3 - \frac{4}{2^n}\bigr)\lvert \omega \rangle
$$

この操作によって $2^n$ 個の振幅のうち $\omega$ が他のよりも約３倍大きくなりました。
以上で振幅増幅させることができました。

## Grover's algorithm
このアルゴリズムでは上の Oracle が与えられたときに $\omega$ を見つけることができます。

内容としては振幅増幅を繰り返し行い、$s$ を $\omega$ に近づけるということをします。

上の式から 1 回振幅増幅を行うと

$$
\cos\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{3}{2}\theta\bigr)\tilde{\lvert \omega \rangle}
$$

となりました。

よって、$n$ 回振幅増幅を行うと

$$
\cos\bigl(\frac{1}{2}\theta + n\theta \bigr)\tilde{\lvert s' \rangle} + \sin\bigl(\frac{1}{2}\theta + n\theta\bigr)\tilde{\lvert \omega \rangle} = \cos\biggl\{\bigl(\frac{1}{2} + n \bigr)\theta \biggr\} \tilde{\lvert s' \rangle} + \sin\biggl\{\bigl(\frac{1}{2} + n \bigr)\theta \biggr\}\tilde{\lvert \omega \rangle}
$$

と表せます。

振幅増幅を行うに連れて上の図の初期状態 $s$ が段々と $\omega$ に近づいて行くことがわかります。
つまり $\omega$ の確率振幅が 1 になっていきます。
ただこのまま繰り返ししても回転し続けるだけなので繰り返す回数を考える必要があります。

$\omega$ の確率振幅を 1 にするので

$$
\sin^2\biggl\{\bigl(\frac{1}{2} + k \bigr)\theta \biggr\} < 1 \xrightarrow{\sin >0}\bigl(\frac{1}{2} + k \bigr)\theta < \frac{\pi}{2} \Leftrightarrow k<\frac{\pi}{2\theta} - \frac{1}{2}
$$

以上から振幅増幅する回数が決まり、$\omega$ が取れることがわかりました。

## 補足 (量子プログラミングへの応用)
量子プログラミングでは振幅増幅は以下のように考えます。

$s$ の定義から $s = s' + \omega$ と分けて考えると、$U_{\omega}$ はシンプルに

$$
U_{\omega} (\lvert s' \rangle + \lvert \omega \rangle) = \lvert s' \rangle - \lvert \omega \rangle
$$

となります。
つまりZ、CZゲートなどのように特定の状態だけ符号を変えるゲートを用いれば良いことがわかります。

$U_s$ は上の図から幾何的に考えると以下のようにかけます。

$$
\begin{align}
U_s U_{\omega} \lvert s \rangle &= 2(\langle s \lvert U_{\omega} \rvert s \rangle \rvert s \rangle - U_{\omega}\lvert s \rangle) + U_{\omega}\lvert s \rangle \\
&= 2\lvert s \rangle \langle s \lvert U_{\omega} \rvert s \rangle - U_{\omega}\lvert s \rangle \\
&= (2\lvert s\rangle \langle s\rvert - I) U_{\omega}\lvert s \rangle
\end{align}
$$

よって $U_s = 2\lvert s\rangle \langle s\rvert - I$ となります。

さらに $U_s$ は以下のように分解できます。

$$
2\lvert s\rangle \langle s\rvert - I = 2H^{\otimes n}\lvert 0^n\rangle \langle 0^n\rvert H^{\otimes n} - I = H^{\otimes n} (2\lvert 0^n\rangle \langle 0^n\rvert - I) H^{\otimes n}\ \ \ \ (\lvert 0^n \rangle = \lvert 00\cdots 00 \rangle)
$$

ここで $2\lvert 0^n\rangle \langle 0^n\rvert - I$ に関して

$$
2\lvert 0^n\rangle \langle 0^n\rvert - I = 
    \begin{pmatrix}
      -1 & 0 & 0 & \ldots & 0 & 0 \\
      0 & 1 & 0 & \ldots & 0 & 0 \\
      0 & 0 & 1 & \ldots & 0 & 0 \\
      \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
      0 & 0 & 0 & \ldots & 1 & 0 \\
      0 & 0 & 0 & \ldots & 0 & 1
    \end{pmatrix}
$$

と表せる。

これは

$$
XZX =
    \begin{pmatrix}
      -1 & 0 \\
      0 & 1
    \end{pmatrix}
$$

の性質から

$$
2\lvert 0^n\rangle \langle 0^n\rvert - I = X^{\otimes n}C^n ZX^{\otimes n}
$$

とかける。

以上から Grover's algorithm をゲートで書き直すことができた。